<a href="https://colab.research.google.com/github/ZamWayne/Machine-Learning-Task/blob/main/Tugas%2012/Azam_A_Google_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama     :Azam Auliyaa

NIM      :1103204205

https://d2l.ai/chapter_convolutional-modern/googlenet.html

In [1]:
!pip install d2l

In [2]:
import torch  # Mengimpor modul torch untuk komputasi tensor
from torch import nn  # Mengimpor modul nn (neural network) dari torch untuk mendefinisikan model neural network
from torch.nn import functional as F  # Mengimpor modul functional dari nn sebagai F untuk fungsi aktivasi dan fungsi lainnya
from d2l import torch as d2l  # Mengimpor modul d2l dari d2l.torch untuk pustaka deep learning yang digunakan

In [3]:
class Inception(nn.Module):  # Mendefinisikan kelas Inception yang merupakan subclass dari nn.Module
    # c1--c4 adalah jumlah saluran output untuk setiap cabang
    def __init__(self, c1, c2, c3, c4, **kwargs):  # Fungsi inisialisasi kelas
        super(Inception, self).__init__(**kwargs)  # Memanggil konstruktor dari superclass
        # Cabang 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)  # Lapisan konvolusi dengan kernel 1x1 untuk cabang 1
        # Cabang 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)  # Lapisan konvolusi dengan kernel 1x1 untuk cabang 2 (bagian pertama)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)  # Lapisan konvolusi dengan kernel 3x3 dan padding 1 untuk cabang 2 (bagian kedua)
        # Cabang 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)  # Lapisan konvolusi dengan kernel 1x1 untuk cabang 3 (bagian pertama)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)  # Lapisan konvolusi dengan kernel 5x5 dan padding 2 untuk cabang 3 (bagian kedua)
        # Cabang 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)  # Lapisan pooling maksimum dengan kernel 3x3, stride 1, dan padding 1 untuk cabang 4
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)  # Lapisan konvolusi dengan kernel 1x1 untuk cabang 4 (bagian kedua)

    def forward(self, x):  # Metode untuk melakukan proses maju
        b1 = F.relu(self.b1_1(x))  # Melakukan konvolusi pada cabang 1 dan menerapkan ReLU
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))  # Melakukan konvolusi pada cabang 2 dan menerapkan ReLU
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))  # Melakukan konvolusi pada cabang 3 dan menerapkan ReLU
        b4 = F.relu(self.b4_2(self.b4_1(x)))  # Melakukan konvolusi pada cabang 4 dan menerapkan ReLU
        return torch.cat((b1, b2, b3, b4), dim=1)  # Menggabungkan hasil dari semua cabang pada dimensi ke-1 (dimensi channel)

In [4]:
class GoogleNet(d2l.Classifier):  # Mendefinisikan kelas GoogleNet yang merupakan subclass dari d2l.Classifier
    def b1(self):  # Mendefinisikan fungsi b1
        return nn.Sequential(  # Mendefinisikan urutan lapisan dalam sebuah Sequential
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),  # Lapisan konvolusi dengan 64 filter, ukuran kernel 7x7, stride 2, dan padding 3
            nn.ReLU(),  # Fungsi aktivasi ReLU
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))  # Lapisan pooling maksimum dengan ukuran kernel 3x3, stride 2, dan padding 1

In [5]:
@d2l.add_to_class(GoogleNet)  # Dekorator yang menambahkan metode b2 ke kelas GoogleNet
def b2(self):  # Fungsi untuk menentukan arsitektur blok b2 dalam jaringan GoogleNet
    return nn.Sequential(  # Membuat urutan lapisan dengan metode Sequential
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),  # Lapisan konvolusi dengan 64 filter, kernel 1x1, dan ReLU
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),  # Lapisan konvolusi dengan 192 filter, kernel 3x3, padding 1, dan ReLU
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))  # Lapisan pooling maksimum dengan kernel 3x3, stride 2, padding 1

In [6]:
@d2l.add_to_class(GoogleNet)  # Menambahkan metode b3 ke kelas GoogleNet
def b3(self):  # Mendefinisikan fungsi b3
    return nn.Sequential(  # Mendefinisikan urutan lapisan dalam sebuah Sequential
        Inception(64, (96, 128), (16, 32), 32),  # Menggunakan lapisan Inception dengan parameter yang ditentukan
        Inception(128, (128, 192), (32, 96), 64),  # Menggunakan lapisan Inception dengan parameter yang ditentukan
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))  # Lapisan pooling maksimum dengan ukuran kernel 3x3, stride 2, dan padding 1



In [7]:
@d2l.add_to_class(GoogleNet)  # Mendekorasi metode b4 agar dapat ditambahkan ke kelas GoogleNet
def b4(self):  # Mendefinisikan metode b4
    return nn.Sequential(  # Mendefinisikan urutan lapisan dalam sebuah Sequential
        Inception(192, (96, 208), (16, 48), 64),  # Inception module dengan parameter yang ditentukan
        Inception(160, (112, 224), (24, 64), 64),  # Inception module dengan parameter yang ditentukan
        Inception(128, (128, 256), (24, 64), 64),  # Inception module dengan parameter yang ditentukan
        Inception(112, (144, 288), (32, 64), 64),  # Inception module dengan parameter yang ditentukan
        Inception(256, (160, 320), (32, 128), 128),  # Inception module dengan parameter yang ditentukan
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))  # Lapisan pooling maksimum dengan ukuran kernel 3x3, stride 2, dan padding 1



In [8]:
@d2l.add_to_class(GoogleNet)
def b5(self):  # Mendekorasi metode b5 untuk ditambahkan ke kelas GoogleNet
    return nn.Sequential(  # Mendefinisikan urutan lapisan dalam sebuah Sequential
        Inception(256, (160, 320), (32, 128), 128),  # Memanggil kelas Inception dengan parameter yang diberikan
        Inception(384, (192, 384), (48, 128), 128),  # Memanggil kelas Inception dengan parameter yang diberikan
        nn.AdaptiveAvgPool2d((1,1)),  # Lapisan pooling adaptif rata-rata dengan ukuran (1,1)
        nn.Flatten())  # Mengubah tensor menjadi vektor

In [9]:
@d2l.add_to_class(GoogleNet)  # Mendekorasi fungsi __init__ agar ditambahkan ke kelas GoogleNet
def __init__(self, lr=0.1, num_classes=10):  # Fungsi inisialisasi kelas dengan parameter lr (learning rate) dan num_classes (jumlah kelas output)
    super(GoogleNet, self).__init__()  # Memanggil konstruktor dari superclass
    self.save_hyperparameters()  # Menyimpan hyperparameter yang diberikan
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(),  # Mendefinisikan urutan lapisan dalam sebuah Sequential
                             self.b5(), nn.LazyLinear(num_classes))  # Menambahkan lapisan akhir dengan jumlah kelas output yang sesuai
    self.net.apply(d2l.init_cnn)  # Mengaplikasikan inisialisasi khusus pada seluruh lapisan dalam jaringan

In [10]:
model = GoogleNet().layer_summary((1, 1, 96, 96)) # Memanggil metode layer_summary() dari objek GoogleNet yang dibuat secara instan.



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
model = GoogleNet(lr=0.01)  # Membuat instance model GoogleNet dengan learning rate 0.01
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)  # Membuat instance trainer dengan maksimum epoch 10 dan menggunakan 1 GPU
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))  # Memuat data FashionMNIST dengan batch size 128 dan ukuran resize 96x96
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)  # Menginisialisasi model menggunakan data pertama dari loader dengan inisialisasi khusus untuk CNN
trainer.fit(model, data)  # Melatih model menggunakan data

100%|██████████| 26421880/26421880 [00:02<00:00, 9016175.86it/s] 


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 342017.33it/s]


Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:03<00:00, 1209326.90it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4252959.82it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
